# Load muData

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import scanpy as sc
import scirpy as ir
import pandas as pd
from mudata import MuData
import muon as mu

In [2]:
%cd /ihome/ylee/yiz133/Code/data_EAE/anndata/

/ihome/ylee/yiz133/Code/data_EAE/anndata


In [22]:
mdata = mu.read("all_Common.h5mu")

airr_DF = ir.get.airr(mdata['airr'], ["cdr3_aa", "umi_count", "v_call", "j_call"] ,  ('VDJ_1', 'VJ_1'))
mdata.obs = mdata.obs.join(airr_DF)
# 'VDJ_1'
mdata

MuData object with n_obs × n_vars = 77660 × 33708
  obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'HTO_classification.global', 'hash.ID', 'percent.mt', 'integrated_nn_res.2', 'seurat_clusters', 'level_2_clusters', 'manual_cell_type', 'run_num', 'sample', 'VDJ_1_cdr3_aa', 'VDJ_1_umi_count', 'VDJ_1_v_call', 'VDJ_1_j_call', 'VJ_1_cdr3_aa', 'VJ_1_umi_count', 'VJ_1_v_call', 'VJ_1_j_call'
  2 modalities
    gex:	77660 x 33708
      obs:	'mouse_id', 'date', 'tissue', 'sample', 'mouse_BC'
    airr:	77660 x 0
      uns:	'chain_indices'
      obsm:	'airr', 'chain_indices'

In [28]:
airr_beta = airr_DF.iloc[:5,:4]

### Analysis

In [29]:
from DeepTCR.DeepTCR import DeepTCR_U

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:

# Instantiate training object
DTCRU = DeepTCR_U('TCR_Analysis')

# Path to your dataset
# data_directory = r"E:\Python code\Machine learning\JupyterNote\Bio\deepTCR\DeepTCR_input"

# Load Data from CNS & Spleen directories
DTCRU.Get_Data(directory=airr_beta, Load_Prev_Data=False, aggregate_by_aa=True,
               aa_column_beta=0, count_column=1, v_beta_column=2, j_beta_column=3)

print("Data loaded successfully into DeepTCR!")


In [ ]:
# Train Variational Autoencoder (VAE) with adjusted parameters
DTCRU.Train_VAE(
    latent_dim=96,          # Reduce latent dimension to prevent overfitting on 20K sequences
    batch_size=5000,         # Reduce batch size to prevent memory overload
    epochs_min=5,            # Ensure at least 5 training epochs
    stop_criterion_window=30, # Increase patience for stopping
    learning_rate=0.001,    # Slightly slower learning rate for better stability
    Load_Prev_Data=False     # Ensure fresh training
)

print("DeepTCR VAE training completed!")


In [ ]:
import pandas as pd

def extract_features_from_DTCRU(DTCRU):
    """
    Extracts latent features along with CDR3 sequences, V-beta, J-beta, and labels into a DataFrame.

    Args:
        DTCRU: The feature_analytics_class instance.

    Returns:
        Pandas DataFrame containing:
        - Features (latent embeddings)
        - CDR3 Beta Sequences
        - V_beta, J_beta
        - Labels (Sample/Cluster ID)
    """

    # Convert latent features to DataFrame
    feature_df = pd.DataFrame(DTCRU.features)

    # Convert sequence data to DataFrame
    metadata_df = pd.DataFrame({
        "CDR3_Beta": DTCRU.beta_sequences,
        "V_Beta": DTCRU.v_beta,
        "J_Beta": DTCRU.j_beta,
        "Label": DTCRU.sample_id  # Use sample_id as labels
    })

    # Merge latent features with metadata
    full_df = pd.concat([metadata_df, feature_df], axis=1)

    return full_df

# Usage:
features_df = extract_features_from_DTCRU(DTCRU)

# Save to CSV if needed
features_df.to_csv("DTCRU_extracted_features_96.csv", index=False)

#features_df 

# Saved embs

In [ ]:
# # Reduce clusters by lowering PhenoGraph resolution (t)
# DTCRU.Cluster(
#     clustering_method='phenograph',  # Keep PhenoGraph clustering
#     t=5,                     # Lower resolution (default ~30); reduce for fewer clusters
#     n_jobs=4                 # Use multiple threads for efficiency
# )
DTCRU.Cluster(
    set='all', 
    clustering_method='dbscan',  # Use DBSCAN
    t=6,  # Higher values merge clusters (default is auto-tuned)
    n_jobs=4
)

In [ ]:
DFs = DTCRU.Cluster_DFs
len(DFs)

In [ ]:
print(DFs[2])

In [ ]:
# import matplotlib.pyplot as plt

# # Initialize lists to store cluster indices and weighted CNS proportions
# cluster_ids = []
# cns_proportions = []

# # Iterate through clusters and compute weighted CNS proportions
# for i, df in enumerate(DFs):
#     total_freq = df["Frequency"].sum()  # Sum of frequencies in cluster
#     cns_freq = df.loc[df["Labels"] == "CNS", "Frequency"].sum()  # Sum of CNS frequencies
    
#     # Avoid division by zero
#     cns_ratio = cns_freq #/ total_freq if total_freq > 0 else 0
    
#     cluster_ids.append(i + 1)  # Cluster numbers start from 1
#     cns_proportions.append(cns_ratio)

# # Plot CNS proportion per cluster
# plt.figure(figsize=(12, 6))
# plt.bar(cluster_ids, cns_proportions, color="dodgerblue")

# plt.xlabel("Cluster")
# plt.ylabel("CNS Proportion (Weighted by Frequency)")
# plt.title("Weighted Proportion of CNS Sequences in Each Cluster")
# #plt.ylim(0, 1)  # Set proportion range between 0 and 1
# plt.xticks(rotation=90)  # Rotate x-axis labels if needed
# plt.show()
# import matplotlib.pyplot as plt

# # Initialize lists to store cluster indices and weighted CNS proportions
# cluster_ids = []
# cns_proportions = []

# # Iterate through clusters and compute weighted CNS proportions
# for i, df in enumerate(DFs):
#     total_freq = df["Frequency"].sum()  # Sum of frequencies in cluster
#     cns_freq = df.loc[df["Labels"] == "Spleen", "Frequency"].sum()  # Sum of CNS frequencies
    
#     # Avoid division by zero
#     cns_ratio = cns_freq #/ total_freq if total_freq > 0 else 0
    
#     cluster_ids.append(i + 1)  # Cluster numbers start from 1
#     cns_proportions.append(cns_ratio)

# # Plot CNS proportion per cluster
# plt.figure(figsize=(12, 6))
# plt.bar(cluster_ids, cns_proportions, color="dodgerblue")

# plt.xlabel("Cluster")
# plt.ylabel("Spleen Proportion (Weighted by Frequency)")
# plt.title("Weighted Proportion of CNS Sequences in Each Cluster")
# #plt.ylim(0, 1)  # Set proportion range between 0 and 1
# plt.xticks(rotation=90)  # Rotate x-axis labels if needed
# plt.show()

In [ ]:
DTCRU.HeatMap_Sequences()

In [ ]:
DTCRU.Structural_Diversity()

In [ ]:
DTCRU.Structural_Diversity_DF

In [ ]:
# Extract normalized frequency and sample IDs from DeepTCR
import numpy as np

normalized_freq = np.array(DTCRU.freq)  # Already normalized per sample
sample_ids = np.array(DTCRU.sample_id)

# Count the number of sequences in CNS and Spleen
num_cns = np.sum(sample_ids == "CNS.tsv")
num_spleen = np.sum(sample_ids == "Spleen.tsv")

# Compute rescaling factor: adjust Spleen frequencies to match CNS scale
scale_factor = num_spleen / num_cns

# Rescale the Spleen frequencies only
adjusted_freq = np.where(sample_ids == "Spleen.tsv", normalized_freq * scale_factor, normalized_freq)
DTCRU.freq = adjusted_freq

In [ ]:
DTCRU.freq

In [ ]:
scale_factor

In [ ]:
num_cns

In [ ]:
num_spleen

In [ ]:
DTCRU.freq

In [ ]:
DTCRU.UMAP_Plot(Load_Prev_Data=True,by_sample=True, show_legend=True,freq_weight=True,scale=10000) #by_cluster=True,by_cluster=True,freq_weight=True

In [ ]:
DTCRU.UMAP_Plot(Load_Prev_Data=True,by_cluster=True, show_legend=False,freq_weight=True,scale=5000)

### Comparison between GIANA embeddings and DeepTCR embeddings

In [ ]:
import pandas as pd

In [ ]:
DTCRU_features_df = pd.read_csv(r"E:\Python code\Machine learning\JupyterNote\Bio\deepTCR\DTCRU_extracted_features_96.csv")

In [ ]:
DTCRU_features_df

In [ ]:
# File paths
embedding_path = r"E:\Python code\Machine learning\JupyterNote\Bio\GIANA\LEE_data\cdr3_only\cdr3_id--RotationEncodingBL62_EncodingMatrix_CFadded.txt"

# Load GIANA embeddings
GIANA_features_df = pd.read_csv(embedding_path, sep="\t", header=None)
GIANA_features_df = GIANA_features_df.iloc[:,1:]
GIANA_features_df = GIANA_features_df.drop(columns=GIANA_features_df.columns[1])
GIANA_features_df = GIANA_features_df.rename(columns={1: 0})
GIANA_features_df

In [ ]:
# for i in range(len(GIANA_features_df)):
#     GIANA_features_df.iloc[i,0] = "C" + GIANA_features_df.iloc[i,0] + "F"

# GIANA_features_df.to_csv('cdr3_id--RotationEncodingBL62_EncodingMatrix_CFadded.csv', header = None, sep="\t")

In [ ]:
import numpy as np
DTCRU_features_df = DTCRU_features_df.iloc[:, np.r_[0, 4:DTCRU_features_df.shape[1]]]
# GIANA_features_df = GIANA_features_df.iloc[:, np.r_[0, 2:GIANA_features_df.shape[1]]]

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

# Step 1: Ensure unique CDR3_Beta sequences by averaging embeddings if duplicated
DTCRU_unique = DTCRU_features_df.groupby("CDR3_Beta").mean().reset_index()
GIANA_unique = GIANA_features_df.groupby(0).mean().reset_index()  # Column 0 contains CDR3_Beta in GIANA

# Step 2: Find the common CDR3 sequences among all three datasets
common_cdr3 = set(DTCRU_unique["CDR3_Beta"]).intersection(GIANA_unique[0])

# Step 3: Keep only common sequences and sort them
DTCRU_matched = DTCRU_unique[DTCRU_unique["CDR3_Beta"].isin(common_cdr3)].sort_values("CDR3_Beta").reset_index(drop=True)
GIANA_matched = GIANA_unique[GIANA_unique[0].isin(common_cdr3)].sort_values(by=0).reset_index(drop=True)

# Step 4: Extract embeddings
DTCRU_embeddings = DTCRU_matched.iloc[:, 1:].values  # Exclude CDR3_Beta (first column)
GIANA_embeddings = GIANA_matched.iloc[:, 1:].values  # Exclude CDR3_Beta (first column)

# Step 5: Compute pairwise distances within each embedding space
DTCRU_distances = squareform(pdist(DTCRU_embeddings, metric="euclidean"))
GIANA_distances = squareform(pdist(GIANA_embeddings, metric="euclidean"))

# Step 6: Flatten upper triangle of the distance matrices for correlation analysis
DTCRU_distances_flat = DTCRU_distances[np.triu_indices_from(DTCRU_distances, k=1)]
GIANA_distances_flat = GIANA_distances[np.triu_indices_from(GIANA_distances, k=1)]



In [ ]:
import numpy as np
from scipy.stats import spearmanr

# Randomly subsample 100,000 pairs for faster correlation
sample_size = min(100000, len(DTCRU_distances_flat))  # Adjust sample size if needed
indices = np.random.choice(len(DTCRU_distances_flat), size=sample_size, replace=False)

# Compute Spearman on subsampled data
spearman_DG, p_DG = spearmanr(DTCRU_distances_flat[indices], GIANA_distances_flat[indices])

results = pd.DataFrame({
    "Comparison": ["DTCRU vs GIANA"],
    "Spearman Correlation": [spearman_DG],
    "p-value": [p_DG]
})
results 

In [ ]:
DTCRU_features_df.shape

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, homogeneity_completeness_v_measure
import pandas as pd

# Perform K-Means clustering for all three datasets
DTCRU_clusters = KMeans(n_clusters=10, random_state=1, n_init=10).fit_predict(DTCRU_embeddings)
GIANA_clusters = KMeans(n_clusters=10, random_state=1, n_init=10).fit_predict(GIANA_embeddings)

# Compute clustering similarity scores for all pairs
ari_DG = adjusted_rand_score(DTCRU_clusters, GIANA_clusters)  # DTCRU vs GIANA

nmi_DG = normalized_mutual_info_score(DTCRU_clusters, GIANA_clusters)

hom_DG, comp_DG, v_DG = homogeneity_completeness_v_measure(DTCRU_clusters, GIANA_clusters)

# Organize results in a DataFrame
clustering_results = pd.DataFrame({
    "Comparison": ["DTCRU vs GIANA"],
    "ARI": [ari_DG],
    "NMI": [nmi_DG],
    "Homogeneity": [hom_DG],
    "Completeness": [comp_DG],
    "V-Measure": [v_DG]
})

clustering_results


In [ ]:
import umap
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Reduce dimensionality using only GIANA embeddings
umap_giana = umap.UMAP(n_components=2).fit_transform(GIANA_embeddings)

# Create dataframe for UMAP representation
df_umap = pd.DataFrame(umap_giana, columns=["UMAP1", "UMAP2"])
df_umap["DTCRU_Cluster"] = pd.Categorical(DTCRU_clusters)  # DTCRU clusters
df_umap["GIANA_Cluster"] = pd.Categorical(GIANA_clusters)  # GIANA clusters

# Set up a 3x1 subplot (3 plots)
fig, axes = plt.subplots(2, 1, figsize=(6, 15))
titles = ["DTCRU Clusters (UMAP GIANA)", "GIANA Clusters (UMAP GIANA)"]

# Define color palettes
dtcru_palette = sns.color_palette("tab10", len(df_umap["DTCRU_Cluster"].unique()))
giana_palette = sns.color_palette("tab10", len(df_umap["GIANA_Cluster"].unique()))

# Plot 1: DTCRU Clusters using GIANA UMAP
sns.scatterplot(data=df_umap, x="UMAP1", y="UMAP2", hue="DTCRU_Cluster", palette=dtcru_palette, alpha=0.6, s=5, ax=axes[0])
axes[0].set_title(titles[0])
axes[0].legend_.remove()

# Plot 2: GIANA Clusters using GIANA UMAP
sns.scatterplot(data=df_umap, x="UMAP1", y="UMAP2", hue="GIANA_Cluster", palette=giana_palette, alpha=0.6, s=5, ax=axes[1])
axes[1].set_title(titles[1])
axes[1].legend_.remove()

plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Create pairwise contingency tables
contingency_DG = pd.crosstab(DTCRU_clusters, GIANA_clusters)

# Function to plot heatmap
def plot_heatmap(contingency_table, xlabel, ylabel, title):
    plt.figure(figsize=(6, 5))
    sns.heatmap(contingency_table, annot=True, fmt="d", cmap="Blues", linewidths=0.5)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

# Plot each heatmap
plot_heatmap(contingency_DG, "GIANA Clusters", "DeepTCR Clusters", "DeepTCR vs GIANA Clusters")
